In [1]:
import torch
import gpytorch as gpt
import botorch
from botorch.models import SingleTaskMultiFidelityGP, SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from botorch.fit import fit_gpytorch_mll, fit_gpytorch_mll_torch
from torch.optim import Adam
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import matplotlib.pyplot as plt
import HeBz
import pickle
from scipy.spatial.distance import cdist

# Use CPU for this example
device = torch.device("cpu")
dtype = torch.float64

/home/sutirthapaul/miniconda3/envs/benzenetest2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Load the data from file
V = np.load('../data/HeBz_sector_2475.npy')
x = np.array([])
y = np.array([])
z = np.array([])
data_points = []
Pot = np.array([])
Pot_diff = np.array([])
for array in V:
    x = np.append(x,array[0])
    y = np.append(y,array[1])
    z = np.append(z,array[2])
    data_points.append([array[0],array[1],array[2],1.0])
    Pot = np.append(Pot,array[3])
V2 = np.load('../data/new_pts_70.npy')
for array in V2:
    x = np.append(x,array[0])
    y = np.append(y,array[1])
    z = np.append(z,array[2])
    data_points.append([array[0],array[1],array[2],1.0])
    Pot = np.append(Pot,array[3])
data_points = np.array(data_points)
noise = 1e-6*np.ones_like(Pot)
print(x)

[4.61939766 3.46410162 3.47733297 ... 4.33012702 4.5        4.        ]


In [4]:
print(np.arange(1,75,2))

[ 1  3  5  7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47
 49 51 53 55 57 59 61 63 65 67 69 71 73]


In [5]:
LJvec = HeBz.LennardJones
Shirvec = HeBz.Shirkov2024
Leevec = HeBz.Lee2003

In [6]:
# -----------------------------
# 1. Generate Source Task Data
# -----------------------------
DFTdata0 = np.load("../data/pbe0_113850_CP_D4_processed.npy")
ind = np.where(np.isclose(DFTdata0[:,2],0.09459459))
refdata = DFTdata0[ind]
refdata[:,2] = -1*refdata[:,2]
DFTdata = np.concatenate((DFTdata0,refdata))
ind = np.argsort(DFTdata[:,2], kind='stable')
DFTdatazsrt = DFTdata[ind]
ind = np.argsort(DFTdatazsrt[:,1], kind='stable')
DFTdatayzsrt = DFTdatazsrt[ind]
DFTdataxrem = DFTdatayzsrt[::8]
ind = np.argsort(DFTdataxrem[:,0], kind='stable')
DFTdataxremzsrt = DFTdataxrem[ind]
DFTdatazxrem = DFTdataxremzsrt
print(DFTdatazxrem)
DFTdata2 = np.load("../data/pbe0_corr_sutirtha_CP_D4_full_processed.npy")
#print(DFTdatazxrem)
#print(np.min(DFTdata2[:,-1]))
DFTdatanew = np.concatenate((DFTdata2[:,0],DFTdata2[:,1],DFTdata2[:,2]),)
DFTtotal = np.concatenate((DFTdatazxrem[:,0:3],DFTdata2[:,0:3]))
data_source = [[x, y, z, w] for x, y, z, w in zip(DFTtotal[:,0],DFTtotal[:,1],DFTtotal[:,2],np.zeros(len(DFTtotal)))]
m_source = np.concatenate((DFTdatazxrem[:,-1],DFTdata2[:,-1]))
noise_source = 1e-6*np.ones_like(m_source)

[[ 0.00000000e+00  0.00000000e+00  9.45945946e-02 ...  1.21158616e-08
  -1.83603042e+04  1.00634447e+05]
 [ 0.00000000e+00  0.00000000e+00  8.51351351e-01 ... -2.11424265e-09
  -8.68586691e+04  4.83971583e+04]
 [ 0.00000000e+00  0.00000000e+00  1.60810811e+00 ...  3.99535650e-10
  -2.31209156e+04  7.70900596e+03]
 ...
 [ 7.00000000e+00  0.00000000e+00  5.39189189e+00 ...  1.37187109e-11
   4.48182311e-01  2.05744395e+00]
 [ 7.00000000e+00  0.00000000e+00  6.14864865e+00 ... -3.75382973e-11
  -2.01439312e-01  2.36286681e+00]
 [ 7.00000000e+00  0.00000000e+00  6.90540541e+00 ...  9.38433445e-13
   9.39158574e-02  2.36453566e+00]]


In [7]:
ind = DFTtotal[:,0] > 4.4
xnew = np.unique(DFTtotal[:,0][ind])
ynew = np.unique(DFTtotal[:,1][ind])
print(len(xnew))

840


In [8]:
ind = np.where(np.isclose(DFTtotal[:,0],4.5))
print(ind)
xpl2 = DFTtotal[:,0][ind]
ypl2 = DFTtotal[:,1][ind]
zpl2 = DFTtotal[:,2][ind]
Potpl2 = m_source[ind]
print(zpl2)

(array([14425, 16654, 16682, 16687, 16696, 16700, 16708, 16712, 16724,
       16730, 16740, 16914, 16940, 16960]),)
[4.12 3.   1.5  0.9  5.   1.8  2.1  7.   2.7  2.4  1.2  0.6  0.3  0.  ]


In [9]:
train_X = np.concatenate((data_points,data_source))
train_Y = np.concatenate((Pot,m_source))
train_Y = train_Y.reshape(-1,1)
train_Yvar = np.concatenate((noise,noise_source))
train_Yvar = train_Yvar.reshape(-1,1)
print(train_X.shape)

(19506, 4)


In [10]:
train_Y_trunc = train_Y[train_Y[:, 0]<=1000]
train_Yvar_trunc = train_Yvar[train_Y[:,0]<=1000]
train_X_trunc = train_X[train_Y[:, 0]<=1000, :]
print(train_X_trunc.shape)

(18162, 4)


In [12]:
state_dict = torch.load('../data/my_model_multfidDFT16kwNoiseRefBetter.pth')
model = SingleTaskMultiFidelityGP(torch.from_numpy(train_X_trunc), torch.from_numpy(train_Y_trunc), torch.from_numpy(train_Yvar_trunc), data_fidelities=[len(train_X_trunc[0]) - 1], input_transform=Normalize(d=4),outcome_transform=Standardize(m=1))
mll = ExactMarginalLogLikelihood(model.likelihood, model)
#Manually remap keys for version compatibility
"""
new_state_dict = {}

for k, v in state_dict.items():
    
    if (k == "likelihood.noise_covar.noise_prior._transformed_loc"):
        new_state_dict["likelihood.noise_covar.noise_prior.concentration"] = v
    elif (k == "likelihood.noise_covar.noise_prior._transformed_scale"):
        new_state_dict["likelihood.noise_covar.noise_prior.rate"] = v
    elif (k == "outcome_transform._is_trained"):
        continue
    else:
        new_key = k.replace("kernels.0.", "")  
        new_state_dict[new_key] = v
"""
model.load_state_dict(state_dict)

/home/sutirthapaul/miniconda3/envs/benzenetest2/lib/python3.11/site-packages/gpytorch/likelihoods/noise_models.py:150: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(


In [13]:
def matern_kernel(x1, x2, lengthscale):
    """
    Compute Matern kernel between x1 and x2 with given lengthscale and smoothness nu.
    """
    # pairwise distance
    r = torch.cdist(x1 / lengthscale, x2 / lengthscale, p=2)
    #print(r)
    sqrt5 = torch.sqrt(torch.tensor(5.0))
    k = (1 + sqrt5 * r + 5 * r**2 / 3) * torch.exp(-sqrt5 * r)
    
    return k
def kernel(X1,X2,theta1,theta2,P,scale):
    # split x and z
    x1, z1 = X1[..., :-1], X1[..., -1:]
    x2, z2 = X2[..., :-1], X2[..., -1:]
    x11_ = z1
    x21t = z2
    x21t_ = x21t.transpose(-1, -2)
    cross_term_1 = (1 - x11_) * (1 - x21t_)
    bias_factor = cross_term_1 * (1 + x11_ * x21t_).pow(P)
    return scale*(matern_kernel(x1,x2,lengthscale=theta1) + bias_factor*matern_kernel(x1,x2,lengthscale=theta2))

In [14]:
ell1 = model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale
ell2 = model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale
p = model.covar_module.base_kernel.kernels[0].power
oscale = model.covar_module.outputscale
learned_mean = model.mean_module.constant
train_x_trunc = torch.from_numpy(train_X_trunc)
train_y_trunc = torch.from_numpy(train_Y_trunc)
xoff = torch.min(train_x_trunc,0)[0]
xscale = torch.max(train_x_trunc,0)[0] -torch.min(train_x_trunc,0)[0]
train_x_nrm = (train_x_trunc - xoff)/xscale
#covar_matrix = kernel(train_x_nrm, train_x_nrm,ell1,ell2,p,oscale)
print(train_x_nrm)
covar_matrix = model.covar_module(train_x_nrm,train_x_nrm).to_dense()
covar_matrix = covar_matrix + model.likelihood.noise* torch.eye(covar_matrix.size(0))
inverse_cov = torch.linalg.inv(covar_matrix)
stdvs = train_y_trunc.std(dim=-2, keepdim=True)
means = train_y_trunc.mean(dim=-2, keepdim=True)
train_y_std = (train_y_trunc - means)/stdvs
#prod = torch.matmul(inverse_cov,train_y_std - learned_mean).to_dense().detach().numpy()   #Store this
prod = torch.linalg.solve(covar_matrix, train_y_std - learned_mean).to_dense().detach().numpy()
print(prod)
#Store all parameters in a dictionary
my_dict = {
    "outputscale": model.covar_module.outputscale.detach().numpy(),
    "theta1": model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale.detach().numpy(),
    "theta2":model.covar_module.base_kernel.kernels[0].covar_module_biased.lengthscale.detach().numpy(),
    "learned_mean": model.mean_module.constant.detach().numpy(),
    "power": model.covar_module.base_kernel.kernels[0].power.detach().numpy(),
    "data":train_x_nrm.numpy(),
    "product": prod,
    "stddevy": stdvs.detach().numpy(),
    "meany": means.detach().numpy(),
    "xoffset": xoff.detach().numpy(),
    "xscale": xscale.detach().numpy()    
}
print(my_dict)
#with open("data.pkl", "wb") as f:
#   pickle.dump(my_dict, f)
np.savez_compressed("data.npz",outputscale= np.array([model.covar_module.outputscale.detach().numpy()]), theta1=model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale.detach().numpy(),
theta2 = model.covar_module.base_kernel.kernels[0].covar_module_biased.lengthscale.detach().numpy(),mean=np.array([model.mean_module.constant.detach().numpy()]),
                   power = np.array([model.covar_module.base_kernel.kernels[0].power.detach().numpy()]), data=train_x_nrm.numpy(),product=prod,
                   stddevy= stdvs.detach().numpy().flatten(), means=means.detach().numpy().flatten(), xoffset=xoff.detach().numpy(), xscale=xscale.detach().numpy())
                    

tensor([[0.6599, 0.5467, 0.5941, 1.0000],
        [0.4949, 0.5714, 0.5941, 1.0000],
        [0.4968, 0.2662, 0.5941, 1.0000],
        ...,
        [0.5939, 0.4920, 0.8647, 0.0000],
        [0.5143, 0.0000, 0.8647, 0.0000],
        [0.6429, 0.0000, 0.0133, 0.0000]], dtype=torch.float64)
[[-2.70225982e+00]
 [ 9.57649686e+00]
 [ 2.78542325e+00]
 ...
 [ 1.73981852e-01]
 [ 1.75306113e+00]
 [-2.07882331e+02]]
{'outputscale': array(814.69663559), 'theta1': array([[0.78638807, 2.17270815, 0.77220716]]), 'theta2': array([[2.03248109, 5.05874827, 1.71032378]]), 'learned_mean': array(22.0553313), 'power': array([0.63911297]), 'data': array([[0.65991395, 0.54669062, 0.59405714, 1.        ],
       [0.49487166, 0.57142857, 0.59405714, 1.        ],
       [0.49676185, 0.26621387, 0.59405714, 1.        ],
       ...,
       [0.59392256, 0.49202156, 0.86468571, 0.        ],
       [0.51428571, 0.        , 0.86468571, 0.        ],
       [0.64285714, 0.        , 0.01333333, 0.        ]], shape=(18162, 